In [1]:
import warnings
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
plt.rcParams['font.sans-serif'] = ['SimHei']  #显示中文
plt.rcParams['axes.unicode_minus']=False #用来正常显示负号
plt.rcParams['figure.dpi'] = 300
%matplotlib inline
# %config InlineBackend.figure_format = 'svg'
warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', None) # 显示所有列
from tqdm.notebook import tqdm
# import geopandas as gpd
# import transbigdata as tbd
# import geoplot as gplt
from TripIdentification import * # 导入自定义函数
import time

In [3]:
import time
# time.sleep(1800)

for i in tqdm(range(9,10)):
    print('读取数据')
    print('读取路径为：  E:/yongqi/mobike/park_to_location_data/park2location_%d.csv'%i)
    print('存储路径为：  E:/yongqi/mobike/location_to_trip_data/loc2trip_%d.csv'%i)

    location_data = pd.read_csv('E:/yongqi/mobike/park_to_location_data/park2location_%d.csv'%i)
    print('location_data.shape:',location_data.shape)
    location_data['timestamp'] = pd.to_datetime(location_data['timestamp'])
    location_data['bikeid'] = location_data['bikeid'].astype('str')
    print('数据类型转换完成')

    location_data['timestamp_next'] = location_data['timestamp'].shift(-1)
    location_data['bikeid_next'] = location_data['bikeid'].shift(-1)
    location_data['time_difference'] = (location_data['timestamp_next'] - location_data['timestamp']).dt.seconds/60
    location_data['geom_lon_next'] = location_data['geom_lon'].shift(-1)
    location_data['geom_lat_next'] = location_data['geom_lat'].shift(-1)
    print('数据shift完成')

    # bikeid条件
    print('location_data.shape:',location_data.shape)
    location_data = location_data[location_data['bikeid'] == location_data['bikeid_next']]
    print('删除不同的bikeid后，location_data.shape:',location_data.shape)
    # 考虑时间阈值
    trip_data = location_data[(location_data['time_difference']>i)&(location_data['time_difference']<120)]
    print('考虑时间阈值后，trip_data.shape:',trip_data.shape)

    # 考虑空间阈值，取100m
    # 曼哈顿距离
    def mhd_dis(lng1, lat1, lng2, lat2):
        '''
        Returns
        ----------
        单位是米
        ''' 
        cos_lat = np.cos(30.656883854354138/180 * np.pi) * 111000 # 米
        lng_dis = cos_lat * abs(lng1-lng2)
        lat_dis = 111000 * abs(lat1-lat2)
        return lng_dis + lat_dis
    trip_data['distance'] = trip_data.apply(lambda x:mhd_dis(x['geom_lon'], x['geom_lat'], x['geom_lon_next'], x['geom_lat_next']),axis=1)
    print('距离计算完成')
    trip_data = trip_data[(trip_data['distance']>100)&(trip_data['distance']<100000)]
    print('考虑空间阈值后，trip_data.shape:',trip_data.shape)

    trip_data.to_csv('E:/yongqi/mobike/location_to_trip_data/loc2trip_%d.csv'%i,index=False)
    print('存储trip_data完成')

  0%|          | 0/1 [00:00<?, ?it/s]

读取数据
读取路径为：  E:/yongqi/mobike/park_to_location_data/park2location_9.csv
存储路径为：  E:/yongqi/mobike/location_to_trip_data/loc2trip_9.csv
location_data.shape: (20218266, 4)
数据类型转换完成
数据shift完成
location_data.shape: (20218266, 9)
删除不同的bikeid后，location_data.shape: (20086205, 9)
考虑时间阈值后，trip_data.shape: (472410, 9)
距离计算完成
考虑空间阈值后，trip_data.shape: (460399, 10)
存储trip_data完成
